## imports

In [1]:
import tensorflow as tf 
import numpy as np
import keras.backend as K

import sys
sys.path.append('/opt/code/face/dual_shot')
import numpy as np
from net.dual_shot import test_net
from prepare_data.generator import image_reader,augment
from prepare_data.augment import Augmentation
from prepare_data.widerface import WIDERFaceDetection
from dual_conf import current_config as conf

config = tf.ConfigProto()
config.gpu_options.allow_growth= True
sess = tf.Session(config=config)
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/root/anaconda3/envs/keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/anaconda3/envs/keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


using: resnet50 as the base model


In [2]:
cls_target = np.random.randint(-1,2,[2,8])
cls_target

array([[-1, -1, -1, -1,  1,  1,  0,  0],
       [-1, -1,  1, -1, -1,  0,  0,  0]])

In [3]:
predict_logits = np.random.randint(-1,10,[2,8,2])
predict_logits

array([[[ 3, -1],
        [ 6,  6],
        [ 9,  2],
        [ 0,  6],
        [ 7,  4],
        [-1,  8],
        [ 8,  6],
        [ 0,  5]],

       [[ 8,  7],
        [ 5,  0],
        [ 5,  7],
        [ 6,  4],
        [ 4,  4],
        [ 0,  6],
        [ 3,  9],
        [ 8,  2]]])

In [4]:
cls_target = tf.constant(cls_target)
predict_logits = tf.constant(predict_logits,tf.float64)

In [5]:
train_indices = tf.where(tf.not_equal(cls_target, 0))
cls_target = tf.gather_nd(cls_target, train_indices)
cls_target = tf.where(cls_target > 0, cls_target, tf.zeros_like(cls_target))
cls_target = tf.one_hot(cls_target, depth=2)

In [6]:
# remove un-trained anchors from pred logit
logit0 = tf.gather_nd(predict_logits[..., 0], train_indices)
logit1 = tf.gather_nd(predict_logits[..., 1], train_indices)
predict_logits = tf.stack([logit0, logit1], axis=1)
predict_logits = tf.cast(predict_logits, dtype=tf.float64)

In [7]:
to_run = [train_indices,cls_target,predict_logits]
for i in to_run:
    print(i,'\n',sess.run(i))

Tensor("Where:0", shape=(?, 2), dtype=int64) 
 [[0 0]
 [0 1]
 [0 2]
 [0 3]
 [0 4]
 [0 5]
 [1 0]
 [1 1]
 [1 2]
 [1 3]
 [1 4]]
Tensor("one_hot:0", shape=(?, 2), dtype=float32) 
 [[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]
Tensor("stack:0", shape=(?, 2), dtype=float64) 
 [[ 3. -1.]
 [ 6.  6.]
 [ 9.  2.]
 [ 0.  6.]
 [ 7.  4.]
 [-1.  8.]
 [ 8.  7.]
 [ 5.  0.]
 [ 5.  7.]
 [ 6.  4.]
 [ 4.  4.]]


In [8]:
# loss_to_rank = log_sum_exp(cls_target,predict_logits)
# to_run = [loss_to_rank]
# for i in to_run:
#     print(i,'\n',sess.run(i))

In [9]:
cls_target,pred_logit = cls_target,predict_logits
logit_max = tf.reduce_max(pred_logit)
loss = tf.log(tf.reduce_sum(tf.exp(pred_logit - logit_max), 1, keepdims=True)) + logit_max
cls_indice = tf.where(cls_target>=1) 
anchor_cls_logit = tf.gather_nd(pred_logit,cls_indice)
loss = tf.squeeze(loss)
loss = loss - anchor_cls_logit


to_run = [pred_logit,cls_target,anchor_cls_logit,loss]
for i in to_run:
    print(i,'\n',sess.run(i))

Tensor("stack:0", shape=(?, 2), dtype=float64) 
 [[ 3. -1.]
 [ 6.  6.]
 [ 9.  2.]
 [ 0.  6.]
 [ 7.  4.]
 [-1.  8.]
 [ 8.  7.]
 [ 5.  0.]
 [ 5.  7.]
 [ 6.  4.]
 [ 4.  4.]]
Tensor("one_hot:0", shape=(?, 2), dtype=float32) 
 [[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]
Tensor("GatherNd_3:0", shape=(?,), dtype=float64) 
 [3. 6. 9. 0. 4. 8. 8. 5. 7. 6. 4.]
Tensor("sub_1:0", dtype=float64) 
 [1.81499279e-02 6.93147181e-01 9.11466454e-04 6.00247569e+00
 3.04858735e+00 1.23402190e-04 3.13261688e-01 6.71534849e-03
 1.26928011e-01 1.26928011e-01 6.93147181e-01]


In [10]:
loss = tf.cast(cls_target[:,0],tf.float64)*loss # set loss for pos anchor to 0
_, neg_ind  = tf.nn.top_k(loss, 2)
pos_ind = tf.squeeze(tf.where(cls_target[:,1]>=1))
pos_ind = tf.cast(pos_ind,tf.int64)
neg_ind = tf.cast(neg_ind,tf.int64)
all_ind = tf.concat([pos_ind,neg_ind],axis=0)
# cls_target = tf.gather(cls_target,all_ind)
# pred_logit = tf.gather(pred_logit,all_ind)
to_run = [all_ind,cls_target,pred_logit,tf.gather(cls_target,all_ind),tf.gather(pred_logit,all_ind)]
for i in to_run:
    print(i,'\n',sess.run(i))

Tensor("concat:0", dtype=int64) 
 [4 5 8 3 1]
Tensor("one_hot:0", shape=(?, 2), dtype=float32) 
 [[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]
Tensor("stack:0", shape=(?, 2), dtype=float64) 
 [[ 3. -1.]
 [ 6.  6.]
 [ 9.  2.]
 [ 0.  6.]
 [ 7.  4.]
 [-1.  8.]
 [ 8.  7.]
 [ 5.  0.]
 [ 5.  7.]
 [ 6.  4.]
 [ 4.  4.]]
Tensor("GatherV2:0", dtype=float32) 
 [[0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]
Tensor("GatherV2_1:0", dtype=float64) 
 [[ 7.  4.]
 [-1.  8.]
 [ 5.  7.]
 [ 0.  6.]
 [ 6.  6.]]


## func

In [11]:
# -*- coding:utf-8 _*-
"""
@author: danna.li
@date: 2019/4/25
@file: loss_func.py
@description:
"""

import tensorflow as tf
import keras.backend as K
from dual_conf import current_config as conf


def log_sum_exp(cls_target, predict_logits):
    """un-averaged confidence loss across all examples in a batch.
    :param cls_target: 2-d tensor,[num_anchors,2 ]; 1,0 for pos,neg anchors respectively
    :param predict_logits: 2-d tensor,[num_anchors,2] 2 for fg and bg
    :return: loss,1-D tensor
    """
    logit_max = tf.reduce_max(predict_logits)
    loss = tf.log(tf.reduce_sum(tf.exp(predict_logits - logit_max), 1, keepdims=True)) + logit_max
    cls_indice = tf.where(cls_target >= 1)
    anchor_cls_logit = tf.gather_nd(predict_logits, cls_indice)
    loss = tf.squeeze(loss)
    loss = loss - anchor_cls_logit
    return loss


def hard_neg_mining(cls_target, predict_logits):
    """
    select top 80 negtive anchors the contribute most loss to do the backward pass
    :param cls_target:2-d tensor,[num_anchors,2 ]; 1,0 for pos,neg anchors respectively
    :param predict_logits: 2-d tensor,[num_anchors,2] 2 for fg and bg
    :return:
    """
    loss_to_rank = log_sum_exp(cls_target, predict_logits)
    loss_to_rank = tf.cast(cls_target[:, 0], tf.float64) * tf.cast(loss_to_rank,
                                                                   tf.float64)  # set loss for pos anchor to 0
    # neg_num = tf.cast(tf.reduce_sum(cls_target[:, -1]) * 3, tf.int32)
    # _, neg_ind = tf.nn.top_k(loss_to_rank, neg_num)
    _, neg_ind = tf.nn.top_k(loss_to_rank, 2)
    pos_ind = tf.squeeze(tf.where(cls_target[:, 1] >= 1))
    pos_ind = tf.cast(pos_ind, tf.int64)
    neg_ind = tf.cast(neg_ind, tf.int64)
    all_ind = tf.concat([pos_ind, neg_ind], axis=0)
    cls_target = tf.gather(cls_target, all_ind)
    predict_logits = tf.gather(predict_logits, all_ind)
    return cls_target, predict_logits


def cls_loss(cls_target, predict_logits):
    """
    :param cls_target:2-d array,[batch_num,num_anchors]; 1,-1,0 for pos,neg and un-train anchors respectively
    :param predict_logits: 3-d array,[batch_num,num_anchors,2] fg or bg
    :return: classification loss of training anchors
    """
    # remove un-trained anchors from cls_target and make cls_target to one-hot format
    train_indices = tf.where(tf.not_equal(cls_target, 0))
    cls_target = tf.gather_nd(cls_target, train_indices)
    # change negative tag from -1 to 0
    cls_target = tf.where(cls_target > 0, cls_target, tf.zeros_like(cls_target))
    cls_target = tf.cast(cls_target, tf.int64)
    cls_target = tf.one_hot(cls_target, depth=conf.num_class)

    # remove un-trained anchors from pred logit
    logit0 = tf.gather_nd(predict_logits[..., 0], train_indices)
    logit1 = tf.gather_nd(predict_logits[..., 1], train_indices)
    predict_logits = tf.stack([logit0, logit1], axis=1)
    # predict_logits = tf.cast(predict_logits, dtype=tf.float32)

    # hard negative anchor mining
    if conf.hard_negative_mining:
        cls_target, predict_logits = hard_neg_mining(cls_target, predict_logits)

    #  calculate the loss
    print(cls_target, predict_logits)
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=cls_target, logits=predict_logits)
    loss = K.mean(loss)
    return loss


if __name__ == '__main__':
    import numpy as np

    cls_target = np.random.randint(-1, 2, [2, 8])
    predict_logits = np.random.randint(-1, 10, [2, 8, 2])
    cls_target = tf.constant(cls_target)
    predict_logits = tf.constant(predict_logits, tf.float64)
    loss = cls_loss(cls_target, predict_logits)
    to_run = [loss]
    sess = tf.Session()
    for i in to_run:
        print(i, '\n', sess.run(i))


Tensor("GatherV2_2:0", dtype=float32) Tensor("GatherV2_3:0", dtype=float64)
Tensor("Mean:0", shape=(), dtype=float64) 
 1.5832593952721088
